#  nlpnet语义角色标注
#### 安装：pip install nlpnet   
#### 国内源安装：pip install nlpnet -i https://pypi.tuna.tsinghua.edu.cn/simple

1.nlpnet是一个基于神经网络的自然语言处理任务的Python库。
目前，它支持词性标注、依存分析以及语义角色标记。

2.首先要下载预训练模型：http://nilc.icmc.usp.br/nlpnet/models.html#srl-portuguese
目前语义角色标注只提供了葡萄牙语的预训练模型

In [2]:
import nlpnet

In [12]:
tagger = nlpnet.SRLTagger('nlpnet-model\srl-pt', language='pt')
sents = tagger.tag(u'O rato roeu a roupa do rei de Roma.')[0]
sents.arg_structures

[('roeu',
  {'A0': ['O', 'rato'],
   'A1': ['a', 'roupa', 'do', 'rei', 'de', 'Roma'],
   'V': ['roeu']})]

#  pyltp语义角色标注

#### pyltp安装有点麻烦-.-，这里记录window 10下的安装方法
1.首先，pip install pyltp安装报错：error: command 'C:\\Program Files (x86)\\Microsoft Visual Studio 14.0\\VC\\BIN\\x86_amd64\\cl.exe' failed with exit status 2

- 安装cmake，下载地址，https://cmake.org/download/ 
- 安装VS2008 EXPRESS，下载网址：https://visualstudio.microsoft.com/zh-hans/vs/express/

2.然后，我选择使用python setup.py install安装
- 下载pyltp，地址：https://github.com/hit-scir/pyltp
- 下载ltp，地址：https://github.com/hit-scir/ltp 
- 解压ltp，然后将解压之后文件命名为ltp，覆盖pyltp文件夹中的ltp
- 打开cmd，进入到pyltp目录下，找到setup.py
- 先执行命令：python setup.py build
- 然后执行命令：python setup.py install

#### 使用里面的预训练模型，需要先下载，然后指定相应目录
- 下载地址：http://ltp.ai/download.html

#### 要先进行分词，词性标注，依存分析

In [15]:
sentence = "我爱自然语言处理技术！"

In [16]:
from pyltp import Segmentor
seg = Segmentor() #生成对象
seg.load("pyltp-model\ltp_data_v3.4.0\cws.model") #加载分词预训练模型
seg_words = seg.segment(sentence)
print(" ".join(seg_words))
seg.release() #释放资源

我 爱 自然 语言 处理 技术 ！


In [17]:
from pyltp import Postagger  
pos=Postagger()
#加载词性预训练模型
pos.load("pyltp-model\ltp_data_v3.4.0\pos.model")
words_pos=pos.postag(seg_words)
for k,v in zip(seg_words, words_pos):
    print(k+'\t'+v)
pos.release()

我	r
爱	v
自然	n
语言	n
处理	v
技术	n
！	wp


In [18]:
from pyltp import Parser
parser=Parser()
parser.load("pyltp-model\ltp_data_v3.4.0\parser.model")
arcs=parser.parse(seg_words,words_pos)
print([(arc.head,arc.relation) for arc in arcs])
parser.release()

[(2, 'SBV'), (0, 'HED'), (4, 'ATT'), (5, 'FOB'), (2, 'VOB'), (5, 'VOB'), (2, 'WP')]


In [19]:
from pyltp import SementicRoleLabeller
labeller = SementicRoleLabeller()
labeller.load("pyltp-model\ltp_data_v3.4.0\pisrl_win.model")
roles = labeller.label(seg_words,words_pos,arcs)
for role in roles:
    print(role.index, "".join(
        ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))
    
labeller.release()

1 A0:(0,0)A1:(2,5)
4 A1:(5,5)
